In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import time
import pickle
from torchvision import datasets, transforms, utils
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize
import multiprocessing
from six.moves import urllib
import multiprocessing as mp
from IPython.core import profiledir
from torch.utils.data import DataLoader, Dataset

opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
from resnet import resnet18, resnet34, resnet50
from densenet import densenet121, densenet161, densenet169
from mobilenetv2 import mobilenet_v2
from googlenet import googlenet
from inception import inception_v3

In [3]:
batch_size_cifar = 50

mean = (0.4914, 0.4822, 0.4465)
std = (0.2471, 0.2435, 0.2616)

t = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize(mean, std),
])

def cifar_loaders(batch_size, shuffle_test=False): 
    data_dir = './data'
    train = datasets.CIFAR10(data_dir, train=True, download=True, 
        transform=t)
    
    test = datasets.CIFAR10(data_dir, train=False, 
        transform=t)
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size,
        shuffle=True, pin_memory=True)
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size,
        shuffle=shuffle_test, pin_memory=True)
    return train_loader, test_loader

train_cifar_loader, test_cifar_loader = cifar_loaders(batch_size_cifar)

Files already downloaded and verified


In [4]:
resnet18_mod = resnet18(pretrained = True).to(device).eval()
resnet34_mod = resnet34(pretrained = True).to(device).eval()
resnet50_mod = resnet50(pretrained = True).to(device).eval()
densenet121_mod = densenet121(pretrained = True).to(device).eval()
densenet161_mod = densenet161(pretrained = True).to(device).eval()
densenet169_mod = densenet169(pretrained = True).to(device).eval()
mobilenet_v2_mod = mobilenet_v2(pretrained = True).to(device).eval()
googlenet_mod = googlenet(pretrained = True).to(device).eval()
inception_v3_mod = inception_v3(pretrained = True).to(device).eval()

In [5]:
torch.cuda.empty_cache()
print(torch.cuda.memory_allocated() / 1024**2)
print(torch.cuda.memory_cached() / 1024**2)

511.00537109375
524.0


/nas/longleaf/home/mikbenjo/.local/lib/python3.9/site-packages/torch/cuda/memory.py:395: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


In [7]:
from sparsemax import Sparsemax
sparsemax = Sparsemax(dim=1)


def true_min_many_images(models,loss,data,label,in_ex,mult_right, tuning_value):
  sz = len(models)
  preds = []
  data.requires_grad = True
  count =0
  for i in range(sz):
      pred = models[i](data)
      if in_ex[i] == 1:
        f_val= loss(pred,label)
      else:
        f_val= tuning_value - mult_right*loss(pred,label)
      preds.append(f_val)
  preds = torch.stack(preds).permute(1,0)
  vector = sparsemax(-preds).data
  final_val = torch.tensordot(preds,vector)
  final_val.backward()
  return data.grad.data

def adversarial_perturbation_in_ex2(data,label,models,loss, lr_d, epsilon, n_iter,in_ex,mult_right,tuning_value):
  sz = len(models)
  delta = torch.zeros(data.size()).to(device)
  mult = 6
  for i in range(n_iter):
    mult = max(mult/2,1)
    data_mod = (data+delta).clamp(min=0,max=1)
    grad = true_min_many_images(models,loss,data_mod,label,in_ex,mult_right, tuning_value)
    delta = (delta +lr_d*torch.sign(grad)).clamp(min=-epsilon,max=epsilon) 
  data_mod = (data+delta).clamp(min=0,max=1)
  del delta

  return data_mod.data

t2 = transforms.Compose([
    # transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

resnet50_mod2 = lambda x: resnet50_mod(t2(x))
densenet169_mod2 = lambda x: densenet169_mod(t2(x))
mobilenet_v2_mod2 = lambda x: mobilenet_v2_mod(t2(x))
inception_v3_mod2 = lambda x: inception_v3_mod(t2(x))

In [8]:
models = [densenet169_mod2, mobilenet_v2_mod2, inception_v3_mod2]

y_n = [-1, -1, 1]
y_n_list_full =  [[-1, -1, 1], [-1, 1, 1], [1, -1, -1], [1, -1, 1], [1, 1, 1], [1, 1, -1], [-1, 1, -1]]
y_n_Judy = [[1, 1, 1], [-1, 1, 1], [-1, -1, 1], [-1, 1, -1]]
y_n_Mimi = [[1, 1, -1], [1, -1, -1], [1, 1, -1]]
y_n_list_test = [[-1, -1, 1]]

criterion = nn.CrossEntropyLoss(reduction="none")

L1 = []
L2 = []

start_total = time.perf_counter()

for combo, y_n in enumerate(y_n_Mimi):
    
    start_combo = time.perf_counter()

    L1 = []
    L2 = []

    for i, (data,label) in enumerate(train_cifar_loader):
        
      if i % 50 == 0:
        print(f"Combo {combo} still in progress")
        
      data = data.to(device)
      label = label.to(device)
      data_mod = adversarial_perturbation_in_ex2(data,label,models,criterion, .005, .0625, 20,y_n,.3,2.5)

      L1.append(data_mod)
      L2.append(label)
    
      del data
      del data_mod
      del label
      torch.cuda.empty_cache()

        
    with open(f"perturbed_imgs_combo_{combo}.pkl", 'wb') as f:
        pickle.dump(L1, f)

    with open(f"labels_combo_{combo}.pkl", 'wb') as f:
        pickle.dump(L2, f)

    finish_combo = time.perf_counter()
    
    print(f'Combo {combo} finished in {round((finish_combo-start_combo)/60, 2)} minutes')

    
finish_total = time.perf_counter()

print(f'Total finished in {round((finish_total-start_total)/60, 2)} minutes')


Combo 0 still in progress
Combo 0 still in progress
Combo 0 still in progress
Combo 0 still in progress
Combo 0 still in progress
Combo 0 still in progress
Combo 0 still in progress
Combo 0 still in progress
Combo 0 still in progress
Combo 0 still in progress
Combo 0 still in progress
Combo 0 still in progress
Combo 0 still in progress
Combo 0 still in progress
Combo 0 still in progress
Combo 0 still in progress
Combo 0 still in progress
Combo 0 still in progress
Combo 0 still in progress
Combo 0 still in progress
Combo 0 finished in 88.56 minutes
Combo 1 still in progress
Combo 1 still in progress
Combo 1 still in progress
Combo 1 still in progress
Combo 1 still in progress
Combo 1 still in progress
Combo 1 still in progress
Combo 1 still in progress
Combo 1 still in progress
Combo 1 still in progress
Combo 1 still in progress
Combo 1 still in progress
Combo 1 still in progress
Combo 1 still in progress
Combo 1 still in progress
Combo 1 still in progress
Combo 1 still in progress
Comb

In [ ]:
# viewing a batch from each combo

for i in range(3):

    file = open(f'perturbed_imgs_combo_{i}.pkl', 'rb')
    images = pickle.load(file)
    file.close()

    file = open(f'labels_combo_{i}.pkl', 'rb')
    labels = pickle.load(file)
    file.close()

    grid = torchvision.utils.make_grid(images[0]).cpu()
    plt.figure(figsize = (16,8))
    plt.imshow(np.transpose(torchvision.utils.make_grid(grid.to(device)[:128], padding=2, normalize=True).cpu(),(1,2,0)))